# This is the Notebook used for cleaning all the features from grid_features.csv File that contains final feaetures from each Grid

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


my_data = pd.read_csv("grid_features.csv")